# API Asos

In [2]:
import pandas as pd
import json
import os
import sys
import dotenv

dotenv.load_dotenv()

sys.path.append(os.path.abspath('../'))
from src import funciones_api as fa
from src import dic

En primer lugar hacemos la llamada a la api para cada una de las marcas obtener todos sus productos:

In [11]:
lista_id = [8264, 4715, 28466, 20758, 18936, 28624]
lista_marcas = ["Mango", "VeroModa", "NAKD", "LoveTriangle", "JadedRose", "Ghospell"]

In [ ]:
fa.busqueda_vestido_marca(lista_id, lista_marcas)

Creamos un diccionario para cada marca donde almacenamos como valor una lista de dos elementos con la lista de ids y el diccionario de precios.

In [12]:
dic_info={}

for marca in lista_marcas:    
    lista_id_vestido = []
    dic_precio = {}

    with open(f"../datos/api_asos/{marca}/vestidos_{marca}.json", 'r') as file:
        response = json.load(file)

    for i in range(len(response["data"]["products"])):
        if "dress" in response["data"]["products"][i]["name"]:
            id = response["data"]["products"][i]["id"]
            lista_id_vestido.append(id)
            dic_precio[id]=response["data"]["products"][i]["price"]["current"]["value"]

    dic_info[marca]=[lista_id_vestido, dic_precio]

Para cada vestido, gacemos una llamada personalizada a la API para obtener más información.

In [9]:
fa.detalles_vestido(lista_marcas, dic_info)

Creamos un diccionario con todo los productos que hemos recolectado dentro de ASOS de las distintas marcas y creamos y almacenamos un dataframe con todos ellos.

In [13]:
dic_vestidos = fa.creacion_diccionario_asos(dic_info, lista_marcas)
df = pd.DataFrame(dic_vestidos)

Mango
VeroModa
NAKD
LoveTriangle
JadedRose
Ghospell


In [14]:
# añadimos la columna de categoria del vestido
df["nombre"] = df["nombre"].str.lower()
df["color"] = df["color"].str.lower()
df["categoria"] = df["nombre"].str.extract('(largo|midi|corto|maxi|mini)', expand=True)

# Como todos los valores de disponible son True, eliminamos la columna
df.drop(columns="disponible", inplace=True)

# Estandarizamos las tallas
df['talla'] = df['talla'].replace(dic.mapo_talla)

In [15]:
df.to_csv("../datos/api_asos/df_asos.csv")
df.to_csv("../datos/dataframes/df_asos.csv")

In [16]:
df.sample(3)

,nombre,marca,precio,color,talla,categoria
804,vestido camisero corto monocromático con detal...,Ghospell,97.5,black,M,corto
469,vestido midi verde oliva con cremallera delant...,VeroModa,87.0,olive,L,midi
683,vestido corto blanco marfil asimétrico con bal...,JadedRose,72.0,ivory,XXS,corto
